# 前言
### 数据源来自[阿里云天池](https://tianchi.aliyun.com/dataset/148605)


## 1、数据理解与处理

In [1]:
import pandas as pd
import numpy as np

data=pd.read_csv('user_action.csv')
data.head()

# 字段            字段说明      提取说明
# user_id         用户标识      抽烟&字段脱敏
# item_id         商品标识      字段脱敏
# behavior_type   行为类型      浏览、收藏、加购物车、购买分别对应值1、2、3、4
# item_category   商品分类标识  字段脱敏
# time            行为时间      精确到小时级别


,user_id,item_id,behavior_type,item_category,time
0,98047837,232431562,1,4245,2014-12-06 02
1,97726136,383583590,1,5894,2014-12-09 20
2,98607707,64749712,1,2883,2014-12-18 11
3,98662432,320593836,1,6562,2014-12-06 10
4,98145908,290208520,1,13926,2014-12-16 21


In [2]:
data.info()# 数据集情况 12256906条，5个字段

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12256906 entries, 0 to 12256905
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        int64 
 1   item_id        int64 
 2   behavior_type  int64 
 3   item_category  int64 
 4   time           object
dtypes: int64(4), object(1)
memory usage: 467.6+ MB


In [3]:
data[data.duplicated()].count() # 有重复数据

user_id          6043527
item_id          6043527
behavior_type    6043527
item_category    6043527
time             6043527
dtype: int64

In [4]:
data.drop_duplicates(keep='last',inplace=True) # 删除重复值

In [5]:
data.isnull().sum() # 没有空值

user_id          0
item_id          0
behavior_type    0
item_category    0
time             0
dtype: int64

## 2、数据分析可视化

### 2.1 流量分析

#### 2.1.1总体流量分析

In [6]:
result={}
result['用户总数']=data['user_id'].nunique()
result['浏览次数']=data[data.behavior_type==1].user_id.count()
result['收藏用户总数']=data[data.behavior_type==2].user_id.nunique()
result['加购物车用户总数']=data[data.behavior_type==3].user_id.nunique()
result['下单用户总数']=data[data.behavior_type==4].user_id.nunique()
result['商品总数']=data['item_id'].nunique()
result['卖出商品总数']=data[data.behavior_type==4].item_id.nunique()
result['类别总数']=data['item_category'].nunique()
result['卖出类别总数']=data[data.behavior_type==4].item_category.nunique()

In [7]:
result

{'用户总数': 10000,
 '浏览次数': 5535879,
 '收藏用户总数': 6730,
 '加购物车用户总数': 8614,
 '下单用户总数': 8886,
 '商品总数': 2876947,
 '卖出商品总数': 92753,
 '类别总数': 8916,
 '卖出类别总数': 4665}

In [8]:
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

table=Table()

headers=['用户总数','浏览次数','收藏用户总数','加购物车用户总数','下单用户总数','下单转换率','商品总数','卖出商品总数','类别总数','卖出类别总数']
rows = [
    [
        result['用户总数'],f"{result['浏览次数']/10000:.2f}万次",result['收藏用户总数'],result['加购物车用户总数'],result['下单用户总数'],
        f"{result['下单用户总数']/result['用户总数']:.2%}",
        f"{result['商品总数']/10000:.2f}万",f"{result['卖出商品总数']/10000:.2f}万",result['类别总数'],result['卖出类别总数'],
    ]
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='整体情况')
)
table.render_notebook()

用户总数,浏览次数,收藏用户总数,加购物车用户总数,下单用户总数,下单转换率,商品总数,卖出商品总数,类别总数,卖出类别总数
10000,553.59万次,6730,8614,8886,88.86%,287.69万,9.28万,8916,4665


#### 2.1.2每日流量分析

In [9]:
data['time']=pd.to_datetime(data['time'])
data['dates']=data.time.dt.date
data['hours']=data.time.dt.hour

pv_day = data[data.behavior_type == 1].groupby('dates')['behavior_type'].count()
uv_day = data[data.behavior_type == 1].groupby('dates')['user_id'].nunique()

In [10]:
attr=list(pv_day.index)
pv=(
    Line()
    .add_xaxis(xaxis_data=attr)
    .add_yaxis("pv",np.around(pv_day.values/10000,decimals=2),label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis(series_name="uv",yaxis_index=1,y_axis=np.around(uv_day.values/1000,decimals=2),label_opts=opts.LabelOpts(is_show=False),)
    .extend_axis(yaxis=opts.AxisOpts(name="uv",type_="value",min_=0,max_=10,interval=2,axislabel_opts=opts.LabelOpts(formatter="{value} 千人"),))
    .set_global_opts(tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type="cross"),
        xaxis_opts=opts.AxisOpts(type_="category",axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),),
        yaxis_opts=opts.AxisOpts(name="pv",type_="value",min_=0,max_=35,interval=7,axislabel_opts=opts.LabelOpts(formatter="{value} 万次"),axistick_opts=opts.AxisTickOpts(is_show=True),splitline_opts=opts.SplitLineOpts(is_show=True),),
        title_opts=opts.TitleOpts(title="pv与uv趋势图",subtitle="按日"),)
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值"),]),
        markarea_opts=opts.MarkAreaOpts(data=[opts.MarkAreaItem(name="双十二", x=("2014-12-11", "2014-12-13")),]))
    )
pv.render_notebook()

##### 结论：从趋势图来看，浏览量和浏览人数在双十二活动期间有一个剧烈的上涨，活动过后趋势又趋于平稳，可利用活动带来的巨大流量，提升用户的购买转化率。

In [11]:
pv_uv = pd.concat([pv_day, uv_day], join='outer', axis=1) 
pv_uv.columns = ['pv_day', 'uv_day']
new_day=pv_uv.diff()
new_day.columns=['new_pv','new_uv']
attr = new_day.index
v = new_day.new_uv
w = new_day.new_pv
#作趋势图
li=(
    Line()
    .add_xaxis(xaxis_data=attr)
    .add_yaxis("新增pv",w,label_opts=opts.LabelOpts(is_show=False))
    .extend_axis(yaxis=opts.AxisOpts(name="新增uv",type_="value",min_=-2000,max_=2000,interval=500,axislabel_opts=opts.LabelOpts(formatter="{value}"),))
    .set_global_opts(tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", axis_pointer_type="cross"),
        xaxis_opts=opts.AxisOpts(type_="category",axispointer_opts=opts.AxisPointerOpts(is_show=True, type_="shadow"),),
        yaxis_opts=opts.AxisOpts(name="新增pv",type_="value",min_=-120000,max_=100000,interval=30000,axislabel_opts=opts.LabelOpts(formatter="{value}"),axistick_opts=opts.AxisTickOpts(is_show=True),splitline_opts=opts.SplitLineOpts(is_show=True),),
        title_opts=opts.TitleOpts(title="pv、uv差异分析"),)
    .set_series_opts(markarea_opts=opts.MarkAreaOpts(data=[opts.MarkAreaItem(name="双十二", x=("2014-12-11", "2014-12-13")),])))
il=(Line()
    .add_xaxis(xaxis_data=attr)
    .add_yaxis("新增uv",v,yaxis_index='1',label_opts=opts.LabelOpts(is_show=False),)
   )
c=li.overlap(il)
c.render_notebook()

##### 结论：双12期间流量波动大


#### 2.1.3每时流量分析

In [12]:
pv_hour=data[data.behavior_type==1].groupby('hours')['behavior_type'].count()
uv_hour=data[data.behavior_type==1].drop_duplicates(['user_id','hours']).groupby('hours')['user_id'].count()
attr=list(pv_hour.index)

pv_h=(
    Line()
    .add_xaxis(xaxis_data=attr)
    .add_yaxis("pv",np.around(pv_hour.values/10000,decimals=2),label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis(series_name="uv",yaxis_index=1,y_axis=np.around(uv_hour.values/1000,decimals=2),label_opts=opts.LabelOpts(is_show=False))
    .extend_axis(yaxis=opts.AxisOpts(name="uv",type_='value',min_=0,max_=10,interval=2,axislabel_opts=opts.LabelOpts(formatter="{value} 千人")))
    .set_global_opts(tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type="cross"),
         xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow")),
         yaxis_opts=opts.AxisOpts(name="pv",type_="value",min_=0,max_=50,interval=10,axislabel_opts=opts.LabelOpts(formatter="{value} 万次"),axistick_opts=opts.AxisTickOpts(is_show=True),splitline_opts=opts.SplitLineOpts(is_show=True)),         
         title_opts=opts.TitleOpts(title="pv与uv趋势图",subtitle="按小时"),          
        ) 
     )
pv_h.render_notebook()

##### 结论：晚上19点到22点是用户最活跃的时段，可以加大推荐用户感兴趣的商品，提高转化率

### 2.2 行为分析

#### 2.2.1用户行为趋势（按日）

In [13]:
collect=data[data.behavior_type==2].groupby('dates')['user_id'].nunique()
shopping_cart=data[data.behavior_type==3].groupby('dates')['user_id'].nunique()
buy=data[data.behavior_type==4].groupby('dates')['user_id'].nunique()
attr_a=list(collect.index)
v_1=collect.values.tolist()
v_2=shopping_cart.values.tolist()
v_3=buy.values.tolist()

c=(
    Line()
    .add_xaxis(attr_a)
    .add_yaxis("收藏人数",v_1,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("加购人数",v_2,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("下单人数",v_3,label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="用户行为数据",subtitle="按日"))
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值")]))
)
c.render_notebook()

#### 2.2.2用户行为趋势（按时）

In [14]:
uv_h=data[data.behavior_type==1].groupby('hours')['user_id'].count()
collect_h=data[data.behavior_type==2].groupby('hours')['user_id'].nunique()
cart_h=data[data.behavior_type==3].groupby('hours')['user_id'].nunique()
buy_h=data[data.behavior_type==4].groupby('hours')['user_id'].nunique()
attr_h=list(uv_h.index)
h1=(uv_h.values//31).tolist()
h2=(collect_h.values//31).tolist()
h3=(cart_h.values//31).tolist()
h4=(buy_h.values//31).tolist()

line=(
    Line()
    .add_xaxis(xaxis_data=attr_h)
    .add_yaxis("收藏人数",h2,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("加购人数",h3,label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("下单人数",h4,label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="日均用户行为数据",pos_top="48%"),legend_opts=opts.LegendOpts(pos_top="48%"))
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值")]))
)
bar=(
    Bar()
    .add_xaxis(xaxis_data=attr_h)
    .add_yaxis("浏览次数",h1,label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="pv对比数据"))
    .set_series_opts(markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max",name="最大值"),opts.MarkPointItem(type_="min",name="最小值")]))
)
ggrid=(
    Grid()
    .add(bar,grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(line,grid_opts=opts.GridOpts(pos_top="60%"))
)
ggrid.render_notebook()

##### 结论：晚上9点是高峰期，可以集中在这个时段进行促销活动


#### 2.2.3转化率

In [15]:
buy_h=data[data.behavior_type==4].drop_duplicates().groupby('hours')['user_id'].count()
collect_h=data[data.behavior_type==2].drop_duplicates().groupby('hours')['user_id'].count()
cart_h=data[data.behavior_type==3].drop_duplicates().groupby('hours')['user_id'].count()
uv_h=data.drop_duplicates().groupby('hours')['user_id'].count()
uv_buy_rate=buy_h/uv_h # 浏览到购买转化率
uv_collect_cart_rate=(cart_h+collect_h)/uv_h  # 浏览到收藏加入购物车的转化率
cart_collect_buy_rate=buy_h/(cart_h+collect_h) # 购物车收藏到购买的转化率

attr_o=list(buy_h.index)
r_1=np.around(uv_buy_rate.values,decimals=2)
r_2=np.around(uv_collect_cart_rate.values,decimals=2)
r_3=np.around(cart_collect_buy_rate.values,decimals=2)

c=(
    Line()
    .add_xaxis(xaxis_data=attr_o)
    .add_yaxis("浏览到购买转化率",r_1)
    .add_yaxis("浏览到收藏加入购物车的转化率",r_2)
    .add_yaxis("购物车收藏到购买的转化率",r_3)
    .set_global_opts(title_opts=opts.TitleOpts(title="不同时段转换率"))
)
c.render_notebook()

#### 2.2.4行为漏斗转化分析

In [16]:
buy_a=data[data.behavior_type==4]['user_id'].count()
collect_a=data[data.behavior_type==2]['user_id'].count()
cart_a=data[data.behavior_type==3]['user_id'].count()
pv_a=data[data.behavior_type==1]['user_id'].count()
attr_a=["浏览","收藏","加入购物车","购买"]
values=[np.around((pv_a/pv_a*100),2),np.around((collect_a/pv_a*100),2),np.around((cart_a/pv_a*100),2),np.around((buy_a/pv_a*100),2)]
data_a=[[attr_a[i],values[i]] for i in range(len(attr_a))]

c=(
    Funnel()
    .add(series_name="用户行为",data_pair=data_a,gap=2,
        tooltip_opts=opts.TooltipOpts(trigger="item",formatter="{a}<br/>{b}:{c}%",is_show=True),
        label_opts=opts.LabelOpts(is_show=True,position="ourside"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff",border_width=1))
    .set_global_opts(title_opts=opts.TitleOpts(title="用户转化漏斗"))
)
c.render_notebook()

##### 结论：在总的浏览量中，有5.99%加入购物车，有4.33%收藏，只有1.93%下单购买，整体来看，购买的转化率最低，有很大的增长空间；单独来看“浏览-加入购物车“这一环节的转化率最低，按照“点击-加入购物车-收藏-购买”这一用户行为路径，我们可通过优化“点击-加入购物车”这一环节进而提升购买的转化率，可以通过鼓励用户收藏加购后可以领优惠券来刺激用户加购收藏从而刺激用户的购买欲望。


### 2.3 商品分析

#### 2.3.1商品销售

In [17]:
cate_pv=data[data.behavior_type==1].groupby('item_category')['item_category'].count()
cate_buy=data[data.behavior_type==4].groupby('item_category')['item_category'].count()
cate_pv_buy=pd.concat([cate_pv,cate_buy],axis=1).fillna(0)
cate_pv_buy.columns=['pv','buy']

c=(
    Scatter()
    .add_xaxis(list(cate_pv_buy.pv))
    .add_yaxis('',list(cate_pv_buy.buy),label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="商品类目浏览与购买关系"),
                    xaxis_opts=opts.AxisOpts(type_='value'),
                    yaxis_opts=opts.AxisOpts(type_='value'))
)
c.render_notebook()

##### 结论：从散点图可得知，大部分商品类目都是浏览量和购买量都比较低，整体呈正相关性。

In [18]:
item_buy=data[data.behavior_type==4].groupby('item_id')['item_id'].count()
i=pd.DataFrame(item_buy)
i.columns=['buy']
item=i.groupby('buy')['buy'].count()
c=(
    Bar()
    .add_xaxis(list(item.index))
    .add_yaxis('商品数',item.values.tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title='商品购买次数'))
)
c.render_notebook()

##### 结论：90%以上的商品只销售了一次，说明商品的销售主要是靠长尾商品，并没有爆款商品的带动。

#### 2.3.2 Top20商品分析

In [19]:
top20_pv=data[data.behavior_type==1].groupby('item_id')['behavior_type'].count().reset_index().sort_values('behavior_type',ascending=False).head(20)
top20_cart=data[data.behavior_type==3].groupby('item_id')['behavior_type'].count().reset_index().sort_values('behavior_type',ascending=False).head(20)
top20_favor=data[data.behavior_type==2].groupby('item_id')['behavior_type'].count().reset_index().sort_values('behavior_type',ascending=False).head(20)
top20_buy=data[data.behavior_type==4].groupby('item_id')['behavior_type'].count().reset_index().sort_values('behavior_type',ascending=False).head(20)
t1_x=top20_pv.index.tolist()
t1_y=top20_pv.behavior_type.tolist()
t2_x=top20_cart.index.tolist()
t2_y=top20_cart.behavior_type.tolist()
t3_x=top20_favor.index.tolist()
t3_y=top20_favor.behavior_type.tolist()
t4_x=top20_buy.index.tolist()
t4_y=top20_buy.behavior_type.tolist()

b2 = (Bar()
      .add_xaxis(t2_x)
      .add_yaxis("加购数",t2_y)
      .set_global_opts(title_opts=opts.TitleOpts(title="加购数前二十商品",pos_top="48%"),
                      legend_opts=opts.LegendOpts(pos_top="48%"),
                      xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)))
     )
b1 = (Bar()
      .add_xaxis(t1_x)
      .add_yaxis("浏览量",t1_y)
      .set_global_opts(title_opts=opts.TitleOpts(title="浏览量前二十商品"),
                      xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)))
     )
      
g = (Grid()
    .add(b1, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(b2, grid_opts=opts.GridOpts(pos_top="60%"))
    )
g.render_notebook()

In [20]:
b4 = (Bar()
      .add_xaxis(t4_x)
      .add_yaxis("购买数",t4_y)
      .set_global_opts(title_opts=opts.TitleOpts(title="购买数前二十商品",pos_top="48%"),
                      legend_opts=opts.LegendOpts(pos_top="48%"),
                      xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)))
      .set_series_opts(itemstyle_opts=opts.ItemStyleOpts(color='#D14667'))
     )
b3 = (Bar()
      .add_xaxis(t3_x)
      .add_yaxis("收藏数",t3_y)
      .set_global_opts(title_opts=opts.TitleOpts(title="收藏数前二十商品"),
                      xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)))
      .set_series_opts(itemstyle_opts=opts.ItemStyleOpts(color='#aed54c'))
     )
g = (Grid()
    .add(b3, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(b4, grid_opts=opts.GridOpts(pos_top="60%"))
    )
g.render_notebook()

##### 结论：可以看到，浏览量前列的商品都没有出现在购买前20的商品中，以及加购和收藏前列的商品也很少出现在购买前20，大部分吸引用户关注度的商品都没有很好的转化为实际效益，这也是导致整体转化率较低的原因。


### 2.4 RFM用户分层

##### 由于数据无消费金额一列，这里只针对R和F做分析。提取用户行为是购买的数据并对数据进行索引为用户ID的透视：

In [21]:
data_buy=data.query('behavior_type==4')
rfm=data_buy.pivot_table(index='user_id',values=['dates','behavior_type'],aggfunc={'dates':'max','behavior_type':'count'})

# 转换出R、F列，根据实际业务需求对R、F进行评级打分：
rfm['R']=-(rfm.dates-rfm.dates.max())/np.timedelta64(1,'D')
rfm.rename(columns={'behavior_type':'F'},inplace=True)
rfm['R_score']=pd.cut(rfm['R'],bins=[0,5,10,15,20,30],labels=[5,4,3,2,1],right=False).astype(float)
rfm['F_score']=pd.cut(rfm['F'],bins=[0,10,20,30,35,45],labels=[1,2,3,4,5],right=False).astype(float)

In [22]:
# 定义根据评分刻画用户标签函数并应用：
def rfm_func(x):
    level=x.apply(lambda x:'1' if x>=0 else '0')
    label=level.R_score+level.F_score
    d={
        '11':'重要价值客户',
        '10':'重要保持客户',
        '01':'重要发展客户',
        '00':'重要挽留客户'
    }
    res=d[label]
    return res
rfm['label']=rfm[['R_score','F_score']].apply(lambda x:x-x.mean()).apply(rfm_func,axis=1)
rfm.head()

,F,dates,R,R_score,F_score,label
user_id,,,,,,
4913,5,2014-12-16,2.0,5.0,1.0,重要保持客户
6118,1,2014-12-17,1.0,5.0,1.0,重要保持客户
7528,6,2014-12-13,5.0,4.0,1.0,重要挽留客户
7591,19,2014-12-13,5.0,4.0,2.0,重要发展客户
12645,8,2014-12-14,4.0,5.0,1.0,重要保持客户


In [23]:
r=rfm.groupby('label')['label'].count()
rx=r.index.tolist()
ry=r.values.tolist()
c=(
    Pie()
    .add('',[list(z) for z in zip(rx,ry)])
    .set_global_opts(title_opts=opts.TitleOpts(title='客户分层'),
                    tooltip_opts=opts.TooltipOpts(is_show=True,formatter='{b}:{d}%'))
)
c.render_notebook()

##### 根据RFM模型，共划分为四类客户，分别为重要保持客户, 重要发展客户, 重要价值客户, 重要挽留客户，其中重要挽留客户占比最高，为34.44%

## 3、结论及建议

#### 本次分析从四个不同的角度提出了问题，下面根据分析数据以及可视化结果给出结论和建议：

### 1、流量分析

#### 在以日为单位的时间尺度下，用户在双十二活动期间比平常时期要更加活跃，建议合理有效的根据时间维度进行拉新促销活动。
#### 平时一天中，大部分用户在19时到23时会比较活跃，其中21到22时，是一天当中最活跃的时段，成交量也是这一时段最高。建议可以在用户活跃的高峰期进行推广以使运营效果最大化，此时使用人数最多，活动容易触达用户，营销活动的形式可以通过促销、拼团、直播等形式进行。

### 2、行为分析

#### 从浏览到有购买意向只有1.93%的转化率，当然有一部分用户是直接购买，但也说明大多数用户以浏览页面为主而购买转化较少，此处为转化漏斗中需要改善和提高的环节。针对提高转化率提出一些建议：
#### 优化电商平台的搜索匹配度和推荐策略，主动根据用户喜好推荐相关的商品，优化商品搜索的准确度和聚合能力，对搜索结果排序优先级进行优化。
#### 在商品详情页的展示上突出用户关注的重点信息，精简信息流的呈现方式，减少用户寻找信息的成本。
#### 优化加入购物车和收藏按键的触达，用户在滑屏时也能方便触达，增加功能使用的次数。

### 3、商品分析

#### 针对浏览量高而销量不高的这部分商品，需要提高的是用户从点击进入商品详情页到最终购买的体验。
#### 商品详情页的实际价格是否相比展示价格偏差过大，有的商家为了吸引用户点击在商品展示页投放的价格具有较强吸引力，但实际价格偏高，在用户心中反而引起反感。
#### 详情页的信息流展示是否合理，是否将用户最想看到的部分置于容易看到的位置，便于信息的获取。
#### 优化商品展示的形式，利用视频等方式给用户更直观的感受，提高照片的美观程度。
#### 评论区评价管理，尤其对于差评区的用户反馈进行认真对待，提高自身服务质量。
#### 通过将畅销商品和非畅销商品进行捆绑销售，当然在做这些活动之前，可以先尝试做 A/B 测试，找到最适合的销售方式和最合适的商品类型捆绑，不断提升转化率。

### 4、用户价值
#### 体系中的重要价值客户，需要重点关注，并且活动投放时需谨慎对待。
#### 对于重要保持客户（忠诚度较高，购买能力不足），适当给予折扣或施行捆绑销售销售政策。
#### 对于重要发展客户，可以通过传递新品，通知店铺活动信息。
#### 对于挽留客户，运营活动可以重点针对这部分用户，提高用户使用产品的频率，可以通过拼团打折、积分兑换等活动唤起用户注意力。

### 5、其他方面
#### 提高留存：淘宝APP的留存相对而言较为稳定，让用户提高、保持使用淘宝电商平台的频率相对而言更加重要。
#### 用户推荐：淘宝本身用户基数庞大，知名度高，个人认为在一二线城市的用户基本已经达到饱和，传播工作需要针对三四线城市的渠道下沉，在这些地区针对用户价格敏感度高的特性开展类似拼多多的拼团转发和打折促销活动，扩大这部分用户的使用率。